<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>👂Phonemes</font>](https://www.kaggle.com/competitions/16oct23hse-phonemes/rules)**. [**Instructions**](https://colab.research.google.com/drive/1owkYjuRGkx050LQnM3b3yTzd0Dr2XbeV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Mounted at /content/drive


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v b16oct23hse-phonemes      # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                     # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: b16oct23hse-phonemes
100% 28.8M/28.8M [00:00<00:00, 110MB/s]
Using competition: b16oct23hse-phonemes
  teamId  teamName            submissionDate       score    
--------  ------------------  -------------------  -------  
11112560  Denis Kulakov       2023-11-10 15:28:37  0.93120  
11094096  Kirill Kiosa        2023-11-11 06:46:28  0.93060  
11212509  Polina Volodchenko  2023-11-10 11:51:18  0.92960  
11222985  Selifanov Tikhon    2023-11-10 16:20:56  0.92920  
11096743  Nikita Mahbub       2023-11-10 18:38:51  0.92840  
11115547  Tanya Tomayly       2023-11-11 03:33:18  0.92680  
11108092  Maxim Smeyanov      2023-11-04 09:58:01  0.92600  
11213122  Y                   2023-11-10 13:48:41  0.92500  
11111161  Artem Zakharov      2023-11-10 08:56:12  0.92060  
11154152  Andrew Shatalov     2023-11-10 08:46:50  0.91960  
11135270  AA                  2023-10-31 20:12:44  0.90680  
11141243  N         

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, xgboost, lightgbm as lgb, librosa
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 1.88 s, sys: 362 ms, total: 2.24 s
Wall time: 3.72 s


In [ ]:
df = pd.read_csv('XY_Phonemes.csv', low_memory=False); df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x249,x250,x251,x252,x253,x254,x255,x256,SpkrID,g
0,11.14,15.17,19.12,18.78,13.79,18.98,20.09,19.16,17.32,19.76,...,7.79,10.95,10.05,8.67,10.84,9.50,7.96,9.04,0,NaN
1,10.84,15.20,19.80,17.82,14.10,17.93,19.54,20.97,16.65,22.41,...,12.95,13.05,11.95,11.07,10.81,9.71,11.57,12.22,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59998,12.15,15.78,21.27,19.61,15.14,21.44,23.24,20.91,17.42,21.22,...,8.82,8.00,7.44,8.32,6.51,8.33,6.83,7.02,436,ao
59999,14.78,14.08,15.70,15.51,15.36,15.52,14.68,15.57,15.48,15.45,...,12.87,13.26,13.16,12.65,13.90,13.10,11.16,13.53,436,sh


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 258 entries, x1 to g
dtypes: float64(256), int64(1), object(1)
memory usage: 118.1+ MB


In [ ]:
# Change string labels to numbers in order of increasing size of the entity (Star < Quasi Star < Galaxy)
# df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None)

In [ ]:
vX = df.query('g!=g').drop('g', axis=1)  # slice a test sample (10K observations)
tXY = df.query('g==g')                   # slice training sample
tX, tY = tXY.drop('g', axis=1), tXY.g    # split into training I/O (60K-10K = 50K observation)

In [ ]:
# aa = tXY[tXY.g=='aa'].set_index('g').iloc[[0],0:255]
# ao = tXY[tXY.g=='ao'].set_index('g').iloc[[0],0:255]
# _ = pd.concat([aa,ao]).T.plot(figsize=(20,3), grid=True, title='log-periodogram vs frequency. See ESL textbook, pp.148-149, Figure 5.5');

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

<font color=green><h3><b>$\alpha$. Build polynomial features</b><h3>

In [ ]:
'''Too long'''
# import pandas as pd
# from scipy.interpolate import UnivariateSpline
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# import numpy as np

# # Assuming tX is your DataFrame

# # Step 1: Extract and Smooth Columns x1 to x256
# feature_columns = [f'x{i}' for i in range(1, 257)]  # Columns x1 to x256
# tX_features = tX[feature_columns]

# # Apply cubic spline smoothing
# for col in tX_features.columns:
#     x = np.arange(len(tX_features[col]))
#     spline = UnivariateSpline(x, tX_features[col], k=3)
#     print(col)
#     tX_features[col] = spline(x)

# # Step 2: Recombine Smoothed Data with SpkrID
# tX_smoothed = pd.concat([tX_features, tX['SpkrID']], axis=1)

# # Step 3: Train LDA Model
# # Prepare target variable (SpkrID) and features
# y = tX_smoothed['SpkrID']
# X = tX_smoothed.drop('SpkrID', axis=1)

# # Train LDA model
# lda = LDA()
# lda.fit(X, y)

# # Optionally, evaluate the model's performance


'Too long'

In [1]:
tXcp = tX
vXcp = vX

NameError: ignored

In [ ]:
tX = tXcp
vX = vXcp

from sklearn.preprocessing import StandardScaler, SplineTransformer
scaler = StandardScaler()
tX = scaler.fit_transform(tX)
vX = scaler.transform(vX)


n = 25000  # validation subsample
vX0, vY0 = tX[:n,:], tY[:n]   # subsample for model selection experiments
tX0, tY0 = tX[-n:,:], tY[-n:] # subsample for model training

# poly = PolynomialFeatures(degree=2)    # add non-linear features (powers and interactions)
# tX0p = poly.fit_transform(tX0)  # create object on training set
# vX0p = poly.transform(vX0)      # apply the same object to test set
# vXp  = poly.transform(vX)       # apply the same object to test set

spltt = SplineTransformer(n_knots=4, degree=3)
tX0 = spltt.fit_transform(tX0)
vX0 = spltt.transform(vX0)
vX = spltt.transform(vX)

<font color=green><h3><b>$\beta$. Fit the model to the training dataset</b><h3>

In [ ]:
m = LDA(solver='lsqr',shrinkage=0.41)
print(m.fit(tX0, tY0).score(vX0, vY0))   # fit the model and show test error

0.93012


In [ ]:
'''Cross validation for finding the optimal shrinkage parameter'''
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.model_selection import cross_val_score, StratifiedKFold

# # Define solvers, shrinkages, and cross-validation setup
# solvers = ['svd', 'lsqr']
# solvers = ['lsqr']
# shrinkages = [0.45,0.46,0.47,0.48,0.49,0.5]  # Include more values as needed
# shrinkages = [0.38, 0.4, 0.42]  # Include more values as needed
# cv = StratifiedKFold(n_splits=5)  # Example of 5-fold stratified cross-validation

# # Iterate and test each combination with cross-validation
# for solver in solvers:
#     for shrinkage in shrinkages:
#         # Only 'lsqr' and 'eigen' support shrinkage
#         if solver in ['svd'] and shrinkage is not None:
#             continue

#         # Create LDA model
#         m = LDA(solver=solver, shrinkage=shrinkage)

#         # Perform cross-validation and calculate mean score
#         scores = cross_val_score(m, tX0, tY0, cv=cv)
#         mean_score = scores.mean()
#         print(f"Solver: {solver}, Shrinkage: {shrinkage}, Mean CV Score: {mean_score}")
# '''Best value for shrinkage is 0.9246'''

# '''
# Solver: lsqr, Shrinkage: 0.38, Mean CV Score: 0.9235200000000001
# Solver: lsqr, Shrinkage: 0.4, Mean CV Score: 0.9235599999999999
# Solver: lsqr, Shrinkage: 0.42, Mean CV Score: 0.9231199999999999
# '''

'Cross validation for finding the optimal shrinkage parameter'

<font color=green><h3><b>$\gamma$. Generate predictions</b><h3>

In [ ]:
pY = pd.DataFrame(m.predict(vX), index=range(1,len(vX)+1), columns=['g'])  # ensure that labels and observations are in corresponding order
pd.DataFrame(df.g.fillna('unknown').value_counts(sort=False)).rename(columns={'g':'observed output g'}).T   # distribution of all train labels
pd.DataFrame(pY.value_counts(sort=False)).rename(columns={0:'predicted output g'}).T
ToCSV(pY, '👂Baseline')

,unknown,aa,ao,dcl,sh,iy
observed output g,10000,7635,11197,8340,9674,13154


g,aa,ao,dcl,iy,sh
predicted output g,1405,2378,1637,2634,1946


<font color=green><h3><b>$\epsilon$. Documentation</b></h3></font>

<font color=green><h4><b>Task 1. Explain Decisions in Preprocessing Pipeline</b></h4></font>

<font color=green>
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.</font>

<font color=green>

1. Why did you choose these elements? (Something in EDA, prior experience,...? Note: EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

 In order to reduce noise and capture underlying patterns in the data we use Spline transformation. After we use Standard scaling to ensure that all features are on the same scale. Also, spline transformation may capture non-linear relationships in the data, enhancing the model's ability to generalize.

<font color=green><h4><b>Task 2. Explain Decisions in Modeling Pipeline</b></h4></font>

<font color=green>
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

LDA is chosen for its ability to find linear combinations of features that best separate different classes, making it suitable for classification tasks. Tuning the shrinkage parameter helps improve the model's performance by regularizing the covariance matrix estimation. Cross-validation provides a robust estimate of how well the model will generalize to unseen data, helping to choose the best combination of parameters and polynomial features might capture non-linear relationships in the data, potentially improving the model's performance.

<font color=green><h3><b>$\zeta$. References</b></h3></font>

<font color=green>
Cite your sources to help your peers learn from these (and to avoid plagiarism claims). At the least, HOML textbook should be cited. Use Google Scholar to draw APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources.

1. google.com
2. kaggle.com
3. scikit-learn.org

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 23 sec


## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try smarter sampling from the training set to avoid using all observations (due to time constraint) and, instead, develop more features
1. Try feature restriction, selection, and regularization (see ESL pp.140-141)
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Try encoding and/or incorporating speaker ID. For example, you could scale/transform each speaker's observations separately.
1. Carefully read the example in [ESL textbook, p.149](https://hastie.su.domains/ElemStatLearn/printings/ESLII_print12.pdf#page=168), check seminar Colab and lecture slides
1. Check difference in distribution between test and train data. You might consider [group k-fold](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) for testing locally or your model will memorize user-phoneme releation, instead of phoneme attributes and class.
1. Learn the domain to develop better features. See ["Penalized discriminant analysis"](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C48&q=Penalized+Discriminant+Analysis&btnG=)
1. Look for training outliers that can affect your model's test performance.
  1. Essentially, you need to find and exclude training [influential points](https://en.wikipedia.org/wiki/Influential_observation), but training outliers might be influential
  1. Brute force with [LOOCV](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#Leave-one-out_cross-validation) is slow, but robust. You might find methods to find each training point's contribution to the prediction and remove rare extreme contributors (since they may not be in the test set)
  1. You can find outliers; then evaluate their effect on model's performance
    1. You can evaluate 1D and 2D histograms to find outliers and
    1. You can cluster observations and find outliers that don't fit the clusters (DBSCAN may be helpful)
    1. You can compress high dimensional space to 1D (with PCA, tSNE, UMAP) and then find outliers with a histogram
 1. Additional info:
    1. Harmonics: [wiki](https://en.wikipedia.org/wiki/Harmonic), [sinusoidal regression](https://www.youtube.com/results?search_query=regression+sinusoidal)
    1. Discrete Fourier Transofrm (DFT): [wiki](https://en.wikipedia.org/wiki/Discrete_Fourier_transform), [Steve Brunton @UW](https://www.youtube.com/watch?v=nl9TZanwbBk), [YouTube search](https://www.youtube.com/results?search_query=Discrete+Fourier+transform)
    1. Spectral density estimation [(wiki)](https://en.wikipedia.org/wiki/Spectral_density_estimation), [YouTube search](https://www.youtube.com/results?search_query=spectral+density+estimation)


